<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/Image_Inpaint_CelebHQ_Diffusions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers datasets accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import cv2
from diffusers import DDPMPipeline, DDPMScheduler
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam
import gc

# --- Memory Optimization Setup ---
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

# --- Reduced Batch Size ---
BATCH_SIZE = 2  # Reduced from 8 to prevent OOM

# --- Dataset with Smaller Resolution ---
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduced resolution
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# --- Simplified Mask Generation ---
def create_irregular_mask(image_size=(128, 128)):
    mask = np.ones(image_size, dtype=np.uint8)
    # Generate random rectangle mask
    x = np.random.randint(0, image_size[0]//2)
    y = np.random.randint(0, image_size[1]//2)
    w = np.random.randint(image_size[0]//4, image_size[0]-x)
    h = np.random.randint(image_size[1]//4, image_size[1]-y)
    cv2.rectangle(mask, (x, y), (x+w, y+h), 0, -1)
    return mask

# --- Model Loading with SafeTensors Handling ---
try:
    model = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
except:
    # Fallback for safetensors issue
    model = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256", use_safetensors=False)

# --- Modified UNet with Gradient Checkpointing ---
original_conv_in = model.unet.conv_in
model.unet.conv_in = nn.Conv2d(4, original_conv_in.out_channels,
                              kernel_size=3, padding=1)
model.unet.enable_gradient_checkpointing()

# Move model to GPU and set to eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.unet.to(device)
model.unet.train()

# --- Memory-Optimized Training Loop ---
def collate_fn(batch):
    images = [transform(img['image'].convert('RGB')) for img in batch]
    return torch.stack(images).to(device, non_blocking=True)

train_loader = DataLoader(
    load_dataset("saitsharipova/CelebA-HQ", split="train"),
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    pin_memory=True
)

optimizer = Adam(model.unet.parameters(), lr=1e-4)

for epoch in range(5):  # Reduced epochs
    for images in train_loader:
        # Generate masks directly on GPU
        masks = torch.stack([
            torch.from_numpy(create_irregular_mask()).float()
            for _ in range(images.size(0))
        ]).unsqueeze(1).to(device, non_blocking=True)

        # Mixed Precision Training
        with torch.cuda.amp.autocast():
            masked_images = images * (1 - masks)
            timesteps = torch.randint(0, 1000, (BATCH_SIZE,), device=device)
            noise = torch.randn_like(masked_images)
            noisy_images = model.scheduler.add_noise(masked_images, noise, timesteps)

            # Forward pass with memory cleanup
            outputs = model.unet(torch.cat([noisy_images, masks], dim=1), timesteps).sample
            loss = nn.functional.mse_loss(outputs, noise)

        # Optimizer steps with gradient scaling
        optimizer.zero_grad(set_to_none=True)  # Reduces memory fragmentation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.unet.parameters(), 1.0)
        optimizer.step()

        # Memory cleanup
        del noisy_images, masks, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

# Save final model with safetensors
model.save_pretrained("inpainting_model", safe_serialization=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

scheduler_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/455M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--google--ddpm-celebahq-256/snapshots/cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


DatasetNotFoundError: Dataset 'saitsharipova/CelebA-HQ' doesn't exist on the Hub or cannot be accessed.